<center>
<h1>Chapter 4</h1>
</center>

<hr>

#1. Machine Learning
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

##I'll try to moviate our dip into machine-learning

Here is my attempt at a backstory. Assume we can build a good algorithm for predicting who will perish and who will survive among Titanic passengers. So we are good at choosing 0 or 1 for the Survived column. Now we invent time travel :) We teleport back in time to when passengers were actually boarding the Titanic's sister ship, the Brittanic, launched after the Titanic sunk. We stand at the gangplank and ask each passenger their name. We plug that into our algorithm and advise them to either (a) board because we predict they will survive or (b) wait until trans-Atlantic passenger flights are invented (because we predict they will perish).

It so happens the Brittanic did sink with loss of life!

I'll let you work on the time-travel paradox on your own :)

In summary, we are using something we know about the past to build an algorithm that will predict the future.

##What type of problem are we solving?

We are fortunate to have the outcomes for the Titanic passengers as captured in the `Survived` column. This column has a more generic name, the *label* column. It labels each row/passenger with the outcome we want to predict. Given we have labels, we can use a type of machine-learning called *supervised learning*. In this course we will focus on supervised learning problems, i.e., we will look at data with a label column.

What if we did not have the Survived column? Somehow the record was lost or no one ever recorded it in the first place. We can still make progress using a style called *unsupervised machine learning*. I'd love to look at this in the course but 10 weeks is not much time.

What if we have just a few examples that are labeled? Maybe we know the outcome of 10 passengers. Typically you need lots of labeled data for supervised learning and 10 is not enough. Are we stuck with unsupervised learning? There have been some cool advances that allow you to kind of combine supervised and unsupervised learning into something called *one-shot* or *few-shot* learning. Way cool. But not sure we can get to it in 10 weeks.

Summary:

* If you have labeled data, and typically lots of it, you can use *supervised* learning methods.

* If you have no labeled data, you can use *unsupervised* learning methods.

* If you have a small amount of labeled data, you can use *one-shot* or *few-shot* learning.

We fall under supervised machine learning: we have labeled passenger data.

#2. The problem of overfitting
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

The way I have the problem set up, we are using Titanic data to make predictions about Brittanic passengers. But we have never seen the Brittanic passengers (unless some Titanic survivors were foolish enough to now board the Brittanic). The problem is that when using supervised learning algorithms, we have the tendency to "memorize" the labeled data we have. So we are really good at predicting Titanic passengers, post hoc. But our algorithms can be fragile when looking at new data, e.g., Brittanic passengers. Memorizing the labeled data falls into a category of problems called *overfitting*. We need to mitigate the overfitting problem.

#Ways to mitigate overfitting

This seems kind of hard. All we have is the Titanic data. We don't know what the Brittanic data will look like. How can we avoid overfitting the Titanic data? How about this? We carve out a piece of the Titanic data and treat it like the Brittanic data. So when we build our prediction algorithm, we ignore the piece we carved out. Once we have built our algorithm, we test it out on the data we held out. This is not perfect, but might give us a more realistic view of the accuracy of our algorithm on new data.

Jargon:

* When we split the Titanic data into two pieces, we call one *traning data* and the other *testing data*. Or training set and testing set.

* The machine learning *algorithms* we will look at in this course include KNN (this chapter) and Naive Bayes (later chapters). We might get to Artifical Neural Nets (ANN). All of these are algorithms. All are prone to overfitting.

* A *model* is an algorithm paired with training data. Once we have the model, we are ready to start predicting, e.g., on the testing data.

#Start with holdout

The approach of carving out of a piece of your data for a testing set is called the *holdout method*. It has the benefit of being simple. It has the drawback of removing (likely valuable) data from your training set. If you are starting with too little labeled data to start with (a typical occurence), now you are subtracting even more. Could lead to *underfitting*! We will discuss remedies to this later. For now, let's get to holdout.

First question: how much to holdout for testing? 50%? Too much? How much you decide to holdout is question you have to answer as a data scientist. For now, we will hold out 30%.

BTW: overfitting leads to what you might hear as *variance errors*. We have too much variance and can get lost in irrelevant details. Underfitting leads to bias errors. We have too few data points and we will be overly-biased by the few we have.

Let's bring in our wrangled data now. In case you were unable to generate your own url, you can use mine: https://docs.google.com/spreadsheets/d/e/2PACX-1vShZZ4EVYQQKWcdQ1JzXxIFj8PER0zk9LwF67YY1BR3iXkWBopwFrGS8-VL1bSACKCRBgOsnY3rAqHU/pub?output=csv

In [0]:
import pandas as pd

In [0]:
#your url from last chapter
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vShZZ4EVYQQKWcdQ1JzXxIFj8PER0zk9LwF67YY1BR3iXkWBopwFrGS8-VL1bSACKCRBgOsnY3rAqHU/pub?output=csv'


In [0]:
letters_table = pd.read_csv(url)



In [0]:
letters_table.head()

,Survived,Name,Length,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,"Braund, Mr. Owen Harris",23,2,1,0,1,1,0,0,1,1,0,0,0,1,2,1,0,0,4,1,0,1,0,1,0,0,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51,2,2,2,1,4,1,3,2,2,1,0,2,2,3,2,0,0,5,3,1,1,0,0,0,2,0
2,1,"Heikkinen, Miss. Laina",22,2,0,0,0,2,0,0,1,4,0,2,1,1,3,0,0,0,0,2,0,0,0,0,0,0,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",44,3,0,1,0,6,1,0,2,1,1,0,5,2,0,0,1,1,2,2,2,2,0,0,0,2,0
4,0,"Allen, Mr. William Henry",24,2,0,0,0,2,0,0,1,2,0,0,4,2,2,0,0,0,2,0,0,0,0,1,0,1,0


#3. Set up training and testing sets
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

###Compute lengths of training and testing sets

I need to use the int function to convert a float to an integer. 

In [0]:
testing_n = int(len(letters_table)/3) #30%
testing_n

297

In [0]:
training_n = len(letters_table) - testing_n
training_n

594

##Next question: what to holdout?

This is an interesting question. Perhaps we want a testing set that mirrors the distribution of the overall table. So if we have 61% perishing, we want the testing set to have the same distribution. This is certainly plausible. Here is one discussion-list explanation of various approaches: https://datascience.stackexchange.com/questions/28331/different-test-set-and-training-set-distribution. For now, we are just going to use random sampling. So we want to choose the testing set **randomly** from the full table. Here is my strategy:

1. Build a new table, shuffled_table, that randomly shuffles the full table.

2. Carve out a piece of shuffled_table for training and a piece for testing. We now have 4 tables: (a) original letters_table, (b) random shuffled_table, (c) training_table, and (d) testing_table.

Let's do it.

###A word about randomness

It gets a little tricky to give you target answers once we start doing random things. Maybe your random answer will not match my random answer. To solve this problem, I am going to ask you to "seed your random number generator" (jargon). If we all use the same seed, we will all get the same random numbers. I know it sounds a little paradoxical, but it is a cool way to make sure you match my results.

To get the seed, we need to bring in another library, *numpy*. For you R fans out there, numpy can be used as the Python analog for many R data manipulation functions. I'd love to get into numpy if we have time.

In [0]:
our_seed = 1234  #if we all use this we should get same random data

In [0]:
import numpy as np  #powerful library for manipulating data
rsgen = np.random.RandomState(our_seed)  #we are only going to use numpy's random number generator for now

In [0]:
shuffled_table = letters_table.sample(frac=1, random_state=rsgen)  #frac=1 means shuffle the entire table as opposed to sampling pieces of it

In [0]:
shuffled_table.head()

,Survived,Name,Length,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
523,1,"Hippach, Mrs. Louis Albert (Ida Sophia Fischer)",47,4,1,2,1,2,1,0,4,5,0,0,2,1,0,2,3,0,3,4,1,1,0,0,0,0,0
778,0,"Kilgannon, Mr. Thomas J",23,2,0,0,0,0,0,1,1,1,1,1,1,2,3,2,0,0,1,1,1,0,0,0,0,0,0
760,0,"Garfirth, Mr. John",18,1,0,0,0,0,1,1,2,1,1,0,0,1,1,1,0,0,3,0,1,0,0,0,0,0,0
496,1,"Eustis, Miss. Elizabeth Mussey",30,1,1,0,0,4,0,0,1,3,0,0,1,2,0,0,0,0,0,6,2,2,0,0,0,1,1
583,0,"Ross, Mr. John Hugo",19,0,0,0,0,0,0,1,2,0,1,0,0,1,1,3,0,0,2,2,0,1,0,0,0,0,0


###We still have the old indices from letters_table

You can see the first row has index 523 instead of 0. I can fix that by reindexing.


In [0]:
shuffled_table = shuffled_table.reset_index(drop=True)  #drop=True says throw away the old index - we don't need it going forward

In [0]:
shuffled_table.head()

,Survived,Name,Length,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,1,"Hippach, Mrs. Louis Albert (Ida Sophia Fischer)",47,4,1,2,1,2,1,0,4,5,0,0,2,1,0,2,3,0,3,4,1,1,0,0,0,0,0
1,0,"Kilgannon, Mr. Thomas J",23,2,0,0,0,0,0,1,1,1,1,1,1,2,3,2,0,0,1,1,1,0,0,0,0,0,0
2,0,"Garfirth, Mr. John",18,1,0,0,0,0,1,1,2,1,1,0,0,1,1,1,0,0,3,0,1,0,0,0,0,0,0
3,1,"Eustis, Miss. Elizabeth Mussey",30,1,1,0,0,4,0,0,1,3,0,0,1,2,0,0,0,0,0,6,2,2,0,0,0,1,1
4,0,"Ross, Mr. John Hugo",19,0,0,0,0,0,0,1,2,0,1,0,0,1,1,3,0,0,2,2,0,1,0,0,0,0,0


That's better. Here it is as a gist if you want to store it.

<pre>
our_seed = 1234  #if we all use this we should get same random data
import numpy as np  #powerful library for manipulating data
rsgen = np.random.RandomState(our_seed)  #we are only going to use numpy's random number generator for now
shuffled_table = letters_table.sample(frac=1, random_state=rsgen)  #frac=1 means shuffle the entire table as opposed to sampling pieces of it
shuffled_table = shuffled_table.reset_index(drop=True)  #drop=True says throw away the old index - we don't need it going forward
</pre>
Creating the testing table is easy peasy. Check it out. I am using slicing, which we have seen, but now using it on the table as opposed to a list. Go, pandas!

In [0]:
testing_table = shuffled_table[:testing_n]  #Just slice off 30% - this makes a copy so shuffled_table unchanged

In [0]:
testing_table = testing_table.reset_index(drop=True)  #make sure we have new indices

In [0]:
training_table = shuffled_table[testing_n:]  #Just slice off last 70% - note subtle change in colon placement

In [0]:
training_table = training_table.reset_index(drop=True)  #make sure we have new indices
training_table.head()

,Survived,Name,Length,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,1,"Simonius-Blumer, Col. Oberst Alfons",35,1,2,1,0,2,1,0,0,2,0,0,3,2,2,4,0,0,2,4,1,2,0,0,0,0,0
1,0,"Sjostedt, Mr. Ernst Adolf",25,1,0,0,2,2,1,0,0,0,1,0,1,1,1,2,0,0,2,3,3,0,0,0,0,0,0
2,0,"Elias, Mr. Tannous",18,2,0,0,0,1,0,0,0,1,0,0,1,1,2,1,0,0,1,2,1,1,0,0,0,0,0
3,0,"Wheadon, Mr. Edward H",21,2,0,0,3,2,0,0,2,0,0,0,0,1,1,1,0,0,2,0,0,0,0,2,0,0,0
4,1,"Newell, Miss. Marjorie",22,1,0,0,0,3,0,0,0,2,1,0,2,2,1,1,0,0,2,2,0,0,0,1,0,0,0


##Rebuild the passengers matrix

Now only want passengers from `training_table`.

So I'll take the answer from Chapter 3, Assignment 2 and modify a bit.



In [0]:
training_passengers_matrix = []  #I am going to use the term "matrix" for a list of lists

for i in range(len(training_table)):
  row = training_table.iloc[i].tolist()
  row_number_list = row[2:]  #3rd item to the end
  training_passengers_matrix.append(row_number_list)

In [0]:
for i in range(5):
  print(training_passengers_matrix[i])

[35, 1, 2, 1, 0, 2, 1, 0, 0, 2, 0, 0, 3, 2, 2, 4, 0, 0, 2, 4, 1, 2, 0, 0, 0, 0, 0]
[25, 1, 0, 0, 2, 2, 1, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 2, 3, 3, 0, 0, 0, 0, 0, 0]
[18, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0]
[21, 2, 0, 0, 3, 2, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0]
[22, 1, 0, 0, 0, 3, 0, 0, 0, 2, 1, 0, 2, 2, 1, 1, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0]


##Grab labels as well

While we are here, let's pull out the Survived column into a list. We have pulled columns into lists before. Maybe you have it as a gist!

In [0]:
training_labels = training_table['Survived'].tolist()  #grab the column and create a list of its values
# this pulls a column of 1's and 0's indicating whether or not the passengers survived

##A preliminary step

Before looking at predicting survival of Titanic passengers, we will need a couple of new functions. I wrote these functions for you. So all you have to do is figure out how to use them!

First, load the  uo_puddles library. You saw this in chapter 3.

<img src='https://www.dropbox.com/s/zv84ew76m7ptyrh/puddles.jpeg?raw=1' height=100>

In [0]:
#flush the old directory
!rm -r  'uo_puddles'

rm: cannot remove 'uo_puddles': No such file or directory


In [0]:
my_github_name = 'uo-puddles'

In [0]:
clone_url = f'https://github.com/{my_github_name}/uo_puddles.git'

In [0]:
#this adds the library to colab so you can now import it
!git clone $clone_url  


Cloning into 'uo_puddles'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 204 (delta 119), reused 64 (delta 33), pack-reused 0
Receiving objects: 100% (204/204), 51.54 KiB | 865.00 KiB/s, done.
Resolving deltas: 100% (119/119), done.


In [0]:
import uo_puddles.uo_puddles as up  #now you have access to cool functions I wrote :)

#4. Now for KNN
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

##How to think about it

I said we can view KNN as a crowdsourcing solution. What is the crowd? It is the list of row-values in `training_passengers_matrix`.

How do we use the crowd? If you give me a passenger, as a 27-element list of numbers, I'll give you an ordered list of passengers closest to that passenger. 

How does the ordered list help me predict whether the passenger will survive or perish? I take the top K from the ordered list, and see if they survived or perished. Each gets to vote one way or the other. Majority wins. If majority says perished, then I predict the passenger will perish. And vice versa.

What is the value of K? It is the "K" in "KNN". It stands for how many experts you want to vote. And you get to choose it! It is not a slam dunk on what the value of K should be. You will have to explore with different values to get the best. And it will change for each new problem you come across.

Jargon alert. K is called a *hyperparameter*. And I would argue that finding the right values for hyperparameters like K is most of the battle in machine learning.



##We actually have done the heavy lifting

Once we have data in correct form, we can call a puddles function to get us going. This function encapsulates what you did yourself in chapter 3 Assignment 3.

Here is a little test. I'll pull first passenger from test set and use training set to get ordered crowd.

In [0]:
test_row_0 = testing_table.iloc[0].tolist()  #pull row from testing table
test_values_0 = test_row_0[2:]
print(test_values_0)

[47, 4, 1, 2, 1, 2, 1, 0, 4, 5, 0, 0, 2, 1, 0, 2, 3, 0, 3, 4, 1, 1, 0, 0, 0, 0, 0]


In [0]:
ordered_crowd = up.ordered_distances_matrix(test_values_0, training_passengers_matrix)  #use training data for matches
ordered_crowd[:5]  #first 5

[(51, 6.324555320336759),
 (154, 6.557438524302),
 (491, 6.6332495807108),
 (396, 6.855654600401044),
 (24, 6.928203230275509)]

##Now choose a value for K

I am going to try 5 to get started. I like odd numbers so I don't have to deal with tie votes.

In [0]:
k = 5

Think I can use the "create one list from another list" gist to get the ids of the top 5 experts.

In [0]:
expert_list = []  #list of row ids from training_table

for i in range(k):
  pair = ordered_crowd[i]  #[index, d]
  index = pair[0]  #just index
  expert_list.append(index)

In [0]:
expert_list

[51, 154, 491, 396, 24]

##How did the experts vote?

In [0]:
expert_votes = []

for i in range(k):
  index = expert_list[i]
  vote = training_labels[index]  #we pulled training_labels out earlier
  expert_votes.append(vote)

In [0]:
expert_votes

[1, 1, 0, 1, 1]

##Majority rules

Looks like 4 votes for survived and 1 for perished.

##I put it all together for you

I packaged up the pieces above into a function. Check it out.

Note I am using a couple of lists we created above:

1. **training_passengers_matrix**. This is a list of rows where each row is a list, itself, representing the length and the counts of the 26 letters.

2. **training_labels**. This is the Survived column in list form.

Both lists should be the same length. They rerpresent different columns in the rows of the training table.

In [0]:
print(len(training_passengers_matrix))
print(len(training_labels))
print(len(training_table))

594
594
594


In [0]:
up.knn(test_values_0, training_passengers_matrix, training_labels, 5, 'euclidean')  #predicting 1 like we expect

1

Did we predict correctly? Let's see.

In [0]:
test_row_0

[1,
 'Hippach, Mrs. Louis Albert (Ida Sophia Fischer)',
 47,
 4,
 1,
 2,
 1,
 2,
 1,
 0,
 4,
 5,
 0,
 0,
 2,
 1,
 0,
 2,
 3,
 0,
 3,
 4,
 1,
 1,
 0,
 0,
 0,
 0,
 0]

In [0]:
test_row_0[0]  #value of Survived for first passenger in testing table

1

###Right at least this time

#5. cosine similarity
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

We have been using Euclidean Distance to measure the similarity of 2 lists of numbers. You can see that in my code above: `up.knn(test_values_0, training_passengers_matrix, training_labels, 5, 'euclidean')`. I designated I wanted to use the 'euclidean' similarity measure. The choice of the similarity measure to use is another *hyperparameter*. Are there alternatives? Yes. And in fact, there is a more preferred measure for dealing with text problems: *cosine similarity*. Let's look at this diagram again.

<img src='https://www.dropbox.com/s/7rtuzw37hgl1oi8/ed_vs_cos.png?raw=1' height=300>

We know that d represents the euclidean distance. But you also see the greek letter theta that measures an angle. We got the angle by drawing 2 lines both starting at (0,0). One line goes to A and the other line goes to B. What I want to do is measure the cosine of the angle (theta) that I get when I draw these lines.

We are now moving into Trigonometry space so have new jargon. The lines are called *vectors*. I find it all kind of interesting. Nothing has really changed on our side. We have 2 lists of numbers. But we can take different mathematical perspectives (Geometry, Trigonometry) to get more abstract views. And this gives us mathematical tools we can use. Cool.

I know you probably have the formula for the cosine of theta sribbled somewhere from your math courses. But just in case, I'll give it to you below.

<img src='https://www.dropbox.com/s/oi1ttx99hf0uejn/cosine.png?raw=1'>

**What are A and B in this formula?** They are 2 lists of 27 values.

Looking at the right-hand side of the formula, **what are A-sub-i and B-sub-i**? In Python, they would be A[i] and B[i].

**What is the greek letter sigma?** We have seen it before. It is summation.
There is one tricky part. You can see that on the sigmas they have i=1, meaning they assume that the lists are indexed 1,2,3, etc. As any sane person would do it. But you know in Python land, you have to have i=0.

I also need to remind you how to take the square-root of a value in Python. There are several ways. But the easiest `x**.5`, i.e., x to the 1/2 power. I am pretty sure I knew this from my high school algebra class :)

The jargony description of the left-hand version of the formula is for the **numerator** we are taking the *dot-product* of 2 vectors and for the **denominator** we are using the *L2 norms* of the 2 vectors (parallel bars around A and B). The formula is nice because you can see how the jargony terms translate into algebra and things we know how to do in pure Python, e.g., loop over lists. I can also tell you that `numpy` knows how to deal with the dot-product and L2 norms directly. So when you import numpy, you bring in another abstraction level, one that knows about *linear algebra* (jargony term for what the formula on left uses for notation).

##Whew.

 Lots of concepts floating around. Trigonometry, vectors, algebra, linear algebra. But in the end, you are doing arithmetic (with some exponentation thrown in) on list of numbers. I'll give you a chance to practice next.

#Stop!
<img src='https://www.dropbox.com/s/kfnmns0f60q8b8z/red_line.png?raw=1' width='300'>

If anyone wants to work as a group, give me a private chat with names. I'll move you all to a break-out room.

If you are in a break-out room and need some help, hit the "request help" button. That pops up a message on my screen. If no one is available, I click later and you can try later. Otherwise I or one of the LAs will join.

If you are in the main meeting and are stuck, send me a private chat and I can move you to an empty room for help.

#Assignment 1
<img src='https://www.dropbox.com/s/3uyvp722kp5to2r/assignment.png?raw=1' width='300'>

I'll give you 2 lists of numbers below. Please compute the cosine from them. I'll break it into pieces for you.

In [0]:
A = test_values_0  #what we have been using, first row in testing table
B = training_passengers_matrix[0]  #first row in training (i.e., crowd) table

In [0]:
[A,B]

[[47,
  4,
  1,
  2,
  1,
  2,
  1,
  0,
  4,
  5,
  0,
  0,
  2,
  1,
  0,
  2,
  3,
  0,
  3,
  4,
  1,
  1,
  0,
  0,
  0,
  0,
  0],
 [35,
  1,
  2,
  1,
  0,
  2,
  1,
  0,
  0,
  2,
  0,
  0,
  3,
  2,
  2,
  4,
  0,
  0,
  2,
  4,
  1,
  2,
  0,
  0,
  0,
  0,
  0]]

Here is formula from above to work from.
<img src='https://www.dropbox.com/s/oi1ttx99hf0uejn/cosine.png?raw=1'>

##Problem 1

Compute the numerator. I used my new list from old list gist, but now have 2 old lists, A and B.

In [0]:
#compute the numerator
numerator_list = []
nfeatures = len(test_values_0)

for i in range(nfeatures):
  Ai = A[i]
  Bi = B[i]
  numerator_list.append(Ai*Bi)

numerator = sum(numerator_list)

In [0]:
numerator  #1709

1709

##A note to the intrepid

I am breaking the denominator into 2 separate loops. If you feel like combining them into a single loop, be my guest :)

I suppose for the truly adventurous, you could build a single loop for the entire computation, both numerator and denominator!

But separate loops are fine, no problem.

##Problem 2

Compute the left-hand side of the denominator. Again, I used new list from old list.

In [0]:
#compute left half of denominator
left_denom_list = []

for i in range(nfeatures):
  left_denom_list.append(A[i]**2)

left_denom = sum(left_denom_list)**.5


In [0]:
left_denom  #48.187135212627034

48.187135212627034

##Problem 3

Compute the right-hand side of the denominator.

In [0]:
#compute right half of denominator
right_denom_list = []

for i in range(nfeatures):
  right_denom_list.append(B[i]**2)

right_denom = sum(right_denom_list)**.5


In [0]:
right_denom  #36.02776706930364

36.02776706930364

##Problem 4

Glue the pieces together.

In [0]:
#glue the pieces together
cosine = numerator/(left_denom*right_denom)


In [0]:
cosine  #0.9844045418547783

0.9844045418547783

##Some things to note about the cosine

As long as all the values in A and B are non-negative, then the cosine will give us a value between 0 and 1.

Unlike euclidean distance, the smaller the cosine value, the more dissimilar the 2 vectors are. The best is a value of 1. The worst is a value of 0.

I wish we had more time to delve into similarity measures. They are what is used to find the similarity among documents, web pages, etc. So there has been a fair amount of work in the area. Here are some pointers if you want to dig a little deeper.

* For text problems there is often similarity between features. For instance, we expect 'q' and 'u' to co-occur. In this case, we might want something slightly more complicated than cosine. The *soft cosine* measure attempts to deal with context in text: [wikipedia](https://en.wikipedia.org/wiki/Cosine_similarity#Soft_cosine_measure) .

* For a broader discussion of the tradeoffs with euclidean distance and cosine similarity, see these: [euclidean-vs-cosine](https://cmry.github.io/notes/euclidean-v-cosine) and [sparse data issues](https://stats.stackexchange.com/questions/29627/euclidean-distance-is-usually-not-good-for-sparse-data-and-more-general-case/30235#30235). 

* More examples of cosine similarity: [see page 121 in this online book](https://nlp.stanford.edu/IR-book/pdf/06vect.pdf).



##I am kind of being mean

I packaged up the cosine similarity function for you. Why did I make you write it from scratch? Good practice :)

In [0]:
up.cosine_similarity(A,B)  #0.9844045418547783, looks the same :)

0.9844045418547783

#Assignment 2
<img src='https://www.dropbox.com/s/3uyvp722kp5to2r/assignment.png?raw=1' width='300'>

I want to evaluate how well we do at predicting the rows in the testing table while using the training table as the crowd. Here is the general idea:

* For each row in the testing table, I want you to call `up.knn` to get a prediction. Go ahead and use this to get a prediction:<br>
 `up.knn(values, training_passengers_matrix, training_labels, 5, 'cosine')`. So hyperparameters are k=5 and sim-measure=cosine.
* I want to add that prediction to a list of predictions I keep.
* When I am done, I will have a list of predictions, one for every row in the testing table.

I'll take over when you are done with that.

In [0]:
#your code
# testing_n

predictions = []
for i in range(testing_n):
  test_values = testing_table.iloc[i].tolist()
  test_values = test_values[2:]
  predictions.append(up.knn(test_values, training_passengers_matrix, training_labels, 5, 'cosine'))

print(predictions)

[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]


In [0]:
#you can copy this code and paste it anywhere you want a message played, e.g., after a long-running loop

import IPython.display as display
display.Audio(url="https://ssl.gstatic.com/dictionary/static/sounds/20180430/complete--_us_1.mp3", autoplay=True)

In [0]:
print(predictions[:20])  #[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0]

[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0]


##On Wednesday, will finish up below

I'm going to use a slick function built into Python called zip. It stands for zipper. Let me give you an example of how it works.

In [0]:
#start with 2 lists of numbers

list_A = [1,2,3]
list_B = [4,5,6]

zipped = list(zip(list_A, list_B))
zipped

[(1, 4), (2, 5), (3, 6)]

##On to the real thing

I am going to use zip to do a zipper on my predictions and the actual labels.

First, I'll get the labels.

In [0]:
testing_labels = testing_table['Survived'].tolist()  #just our standard way of grabbing a column

Now I will zip.

In [0]:
cases = list(zip(predictions,testing_labels))
print(cases[:10])  #[(1, 1), (1, 0), (0, 0), (1, 1), (0, 0), (1, 0), (0, 0), (1, 0), (0, 1), (0, 0)]

[(1, 1), (1, 0), (0, 0), (1, 1), (0, 0), (1, 0), (0, 0), (1, 0), (0, 1), (0, 0)]


You can kind of eyeball this to see which we got right and which wrong. When I see a (1, 1) or a (0, 0), I know my prediction matched the actual value. Give myself a point!

#6. The Confusion Matrix
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

You typically want to know more than simply how many you got right. I hope you see there are 4 cases to consider. Look at the values of first 10 items in cases above. You see:

* (1, 1): this is called True positive. It says we predicted 1 and it was 1.
* (0, 0): this is called True negative. It says we predicted 0 and it was 0.
* (1, 0): this is called False positive. It says we predicted 1 but it was 0.
* (0, 1): this is called False negative. It says we predicted 0 but it was 1.

Our overall accuracy is True positives plus True negatives divided by the length of the testing table. I can use a built-in function, count, to find all 4 cases.


In [0]:
print('True positive: ', cases.count((1,1)))  #True positive:  69
print('True negative: ', cases.count((0,0)))  #True negative:  116
print('False positive: ', cases.count((1,0))) #False positive:  68
print('False negative: ', cases.count((0,1))) #False negative:  44

True positive:  69
True negative:  116
False positive:  68
False negative:  44


In [0]:
(69+116)/testing_n  #0.622895622895623

0.622895622895623

##Beyond accuracy

What I'd like you to consider is some cost overlaid with the 4 cases. In particular, what about the errors we make? For the Brittanic, a false positive tells a passenger that they will survive so go ahead and board. And then they perish. Perhaps we want to minimize false positives, even if it means losing overall accuracy. How would I do that? The only things I have to play with at the moment are (a) the value of K, and (b) the similarity measure, euclidean or cosine. We could play around with different values of those to move around values in the 4 cases.

I see another place we could look. At the moment, we are using majority rule for voting. But that washes out how close the vote was. If all 5 experts vote the same, then that is pretty dang certain. If it is a 3/2 split, then marginally certain. Perhaps we can add a certainty factor into our calculations. Then what? I suppose I could say that for positive predictions, I want a certainty above just 3/2. I might sleep better if I used this to advise Brittanic passengers.


##The 4 cases have been organized

The table below, called a Confusion Matrix, defines a whole set of new measures that go beyond accuracy. The way you use it is you first compute the values of the 4 cases, just like we have done, and then plug them into the centrish 4 boxes. Off of these boxes are new measures that researchers have argued for. Each new measure has a slightly different slant on the proportion of values in the 4 cases.



<img src='https://www.dropbox.com/s/zubecbzi8zsdzgg/confusion_matrix.png?raw=1'>

##Take F1 score as an example

The F1 score is currently popular in computer science publications. It is on the bottom right of the matrix. To compute the F1 score, you take 2/(1/Recall + 1/Precision). That's fine, but where do Recall and Precision come from? From the matrix! Recall is below False negative and Precision is on the right of False positive.

Let's take Precision. It says first get the sum of the true positives. I already have that value. It is 69. Then divide 69 by the number of times I predicted positive, right or wrong. So that is the sum of true-positive and false-postive, or 137. So the precision measure, given the values for my 4 cases, is 69/177.

In [0]:
69/177  #precision value

0.3898305084745763

#Assignment 3
<img src='https://www.dropbox.com/s/3uyvp722kp5to2r/assignment.png?raw=1' width='300'>

Go ahead and compute the F1 score for the 4 case values we have.

In [0]:
recall = cases.count((1,1))/testing_labels.count(1)
recall  #out of all the actual 1 cases, what percentage did I catch
# the proportion of positive cases that we correctly identify

0.6106194690265486

In [0]:
precision = cases.count((1,1))/(cases.count((1,1)) + cases.count((1,0)))
precision  #out of all my 1 guesses, what percentage did I get correct
# the proportion of positive cases that we identified that were correct

0.5036496350364964

In [0]:
f1 = 2/(1/recall + 1/precision)
f1  #0.5519999999999999

0.5519999999999999

##Big question: how to interpret these scores?

It's all rather subjective. It kind of depends on the research field and who is reviewing the quality of your results (e.g., a conference or journal committee, your funding agency). In data science, the F1 score seems popular. But since the matrix above was published, others have argued for yet better measures. And it all comes down to teasing apart the 4 cases in new and interesting ways.

##BTW: were you surprised?

We got 62% accuracy using the length and letter-counts of a passenger's name to predict. It's roughly what we would get by just always guessing 0 (perished). Not a surprise, right? A bit crazy to believe we could glean survival info from length and character counts. In other clasess I have taught using the Titanic data, we have come close to 85% accuracy using the columns we dropped, e.g., Age, Gender, Pclass.

So why are we using just the name? It gets us a step closer to our next Machine Learning algorithm, Naive Bayes. And I hope you can see that KNN does not care what columns we use. All it wants is a list of numbers for each row.

#7. Wrap-up

This ends our first look at KNN. Given time, we may return to it. It is a popular machine-learning algorithm because of its relative simplicity. You can take a table of numeric data and convert a row into a list of values, typically called a vector. And then convert all the vectors into a matrix. Given a new vector, we can find the similarity of the new vector to all the existing ones in the matrix (the crowd). And then take the K most similar as our experts. They get to vote and we use that result as a prediction.

Its downfall is its slowness. We were working with a crowd of roughly 600 rows. What if I had 100,000 rows? And what if the length of each vector was closer to 1000 as opposed to 27? It would be too slow to use practically. You will bump into this on your midterm take-home.

##Ideas we will carry forward

The use of a holdout set to address overfitting will be a theme going forward.

The transformation of text into a numeric form will also be a challenge we will continue to face.

And finally, I want to return to the "cost of being wrong" and how it might effect societal values, biases, etc.